In [2]:
 from __future__ import print_function
import tensorflow as tf
import os
import import_ipynb
from skimage.transform import resize
from skimage.io import imsave
from data import load_test_data , load_train_data

import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose,Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.activations import softmax
 
 
 
 
K.set_image_data_format('channels_last')  # TF dimension ordering in this code
 
img_rows = 96
img_cols = 96
 
smooth = 1.
 
 
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    
 
 
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
 
 
def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    print("++++++++++++++++++++++++++++++++++++++++++++++new Line")
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
 
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
 
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
 
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    print(tf.shape(conv4))
    #split
    td=conv4
    
    tx=softmax(td,axis=[1,2])
    q=1.5
    
    
    
    safe_x = K.maximum(tx,1e-6)
    new_p = K.switch(K.equal(q,1.), K.log(safe_x),(K.pow(safe_x,q))/(q-1))
    new_p = -new_p #-p^q/(q-1)
 
 
    new_p= 1+new_p
    print("new_p",new_p.shape)
    xyz=K.max(new_p,axis=[1,2],keepdims=True)
    print("xyz=",xyz.shape)
   
    
    print("xyz=",xyz.shape)
    weights=1-(new_p/xyz)
    print("weights",weights.shape)
    improved_p=td*weights
    print("improved_p=",improved_p.shape)
    improved_p=K.sum(improved_p,axis=[1,2])/144
    print("improved_pv2=",type(improved_p),improved_p.shape)
    
#     n=tx.shape[0]
#     temp=tf.zeros([0,0],tf.float32)
    
#     for i in range(n):
#         print("=========inside loop",i)
#         safe_x = K.maximum(tx[i],1e-6)
#         new_p = K.switch(K.equal(q,1.), K.log(safe_x),(K.pow(safe_x,q))/(q-1))
#         new_p = -new_p #-p^q/(q-1)
    
    
#         new_p= 1+new_p
#         weights=1-(new_p/K.max(new_p))
    
#         improved_p=td[i]*weights
        
#         temp=tf.concat([temp,(K.sum(improved_p)/144)],0)
    
    
    
    
#     dense_nodes=K.reshape(K.variable(temp),shape=(1,256))
#     print(dense_nodes.shape)
    ####
#     den=[]
#     for i in range(n):
#         den.append(tf.reduce_sum(before_gap[i])/144)
    
    
#     dense_nodes=tf.Variable(den)
#     dense_nodes=tf.reshape(dense_nodes,shape=(1,256))
    
#     dense_nodes=K.sum(before_gap,axis=1)/144.0
#     print(dense_nodes.shape)
#     dense_nodes=K.reshape(before_gap,shape=(1,256))
    
    
#     print("temp type=",type(temp[0]))
#     dense_nodes = tf. convert_to_tensor(improved_p)
#     print("dense node=",dense_nodes)
#     xy=tf.reshape(dense_nodes,(1,256))
#     print("xy=",xy)
    #improved_p=tf.reshape(improved_p,(1,256))
    dense1=Dense(32,activation='relu')(improved_p)
    print("dense1",dense1)
    dense2=Dense(1,activation='softmax')(dense1)
    print("below dense",dense2)
    
    
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    
    print("conv5=",conv5)
 
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
 
    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
 
    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
 
    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
 
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    print("conv10=", conv10) 
 
    model = Model(inputs=[inputs], outputs=[conv10,dense2])
 
    model.compile(optimizer=Adam(lr=1e-5), loss=[dice_coef_loss,
                                                 tf.keras.losses.BinaryCrossentropy()],
                  metrics=[dice_coef,tf.keras.metrics.BinaryCrossentropy()])
 
    return model
 
 
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)
 
    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p
 
 
def train_and_predict():
    imgs_train,imgs_mask_train,imgs_value_train=load_train_data()
 
    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)
 
    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization
 
    imgs_train -= mean
    imgs_train /= std
 
    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]
 
    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss' , mode = 'min' , verbose = 1 , save_best_only=True )
 
    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(imgs_train, y=[imgs_mask_train,imgs_value_train], batch_size=32, epochs= 20, verbose=1, shuffle=True,
              validation_split=0.2 ,  callbacks=[model_checkpoint])
    
    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    imgs_test, imgs_id_test = load_test_data()
    imgs_test = preprocess(imgs_test)

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('weights.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    n_imgs_test = imgs_test.shape[0]
    #for i in range(n_imgs_test):

    #worked for 7 vs 1
    #imgs_mask_test, imgs_value_test= model.predict(np.expand_dims(imgs_test[0] , axis = 0), verbose=1)


    imgs_mask_test, imgs_value_test= model.predict(imgs_test, verbose=1)
    print("mask final img 1" , imgs_mask_test)
    np.save('imgs_mask_test.npy', imgs_mask_test)

    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)
    pred_dir = 'preds'
    
    # print(imgs_mask_test.shape , imgs_value_test.shape)
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
    for image, image_id in zip(imgs_mask_test, imgs_id_test):
        image = (image[:, :, 0] * 255.).astype(np.uint8)
        imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)


 
     
if __name__ == '__main__':
    train_and_predict()

importing Jupyter notebook from data.ipynb
------------------------------
Creating training images...
------------------------------
Done: 0/5635 images
Done: 100/5635 images
Done: 200/5635 images
Done: 300/5635 images
Done: 400/5635 images
Done: 500/5635 images
Done: 600/5635 images
Done: 700/5635 images
Done: 800/5635 images
Done: 900/5635 images
Done: 1000/5635 images
Done: 1100/5635 images
Done: 1200/5635 images
Done: 1300/5635 images
Done: 1400/5635 images
Done: 1500/5635 images
Done: 1600/5635 images
Done: 1700/5635 images
Done: 1800/5635 images
Done: 1900/5635 images
Done: 2000/5635 images
Done: 2100/5635 images
Done: 2200/5635 images
Done: 2300/5635 images
Done: 2400/5635 images
Done: 2500/5635 images
Done: 2600/5635 images
Done: 2700/5635 images
Done: 2800/5635 images
Done: 2900/5635 images
Done: 3000/5635 images
Done: 3100/5635 images
Done: 3200/5635 images
Done: 3300/5635 images
Done: 3400/5635 images
Done: 3500/5635 images
Done: 3600/5635 images
Done: 3700/5635 images
Done:

Epoch 7/20
141/141 [==============================] - ETA: 0s - loss: 8.5043 - conv2d_18_loss: -0.4294 - dense_1_loss: 8.9337 - conv2d_18_dice_coef: 0.4295 - conv2d_18_binary_crossentropy: 0.1677 - dense_1_dice_coef: 0.5891 - dense_1_binary_crossentropy: 8.9337
Epoch 00007: val_loss improved from 8.67694 to 8.63747, saving model to weights.h5
141/141 [==============================] - 668s 5s/step - loss: 8.5043 - conv2d_18_loss: -0.4294 - dense_1_loss: 8.9337 - conv2d_18_dice_coef: 0.4295 - conv2d_18_binary_crossentropy: 0.1677 - dense_1_dice_coef: 0.5891 - dense_1_binary_crossentropy: 8.9337 - val_loss: 8.6375 - val_conv2d_18_loss: -0.4417 - val_dense_1_loss: 9.0792 - val_conv2d_18_dice_coef: 0.4379 - val_conv2d_18_binary_crossentropy: 0.1534 - val_dense_1_dice_coef: 0.5789 - val_dense_1_binary_crossentropy: 9.0792
Epoch 8/20
141/141 [==============================] - ETA: 0s - loss: 8.4853 - conv2d_18_loss: -0.4485 - dense_1_loss: 8.9337 - conv2d_18_dice_coef: 0.4485 - conv2d_18_bin

Epoch 17/20
141/141 [==============================] - ETA: 0s - loss: 8.3868 - conv2d_18_loss: -0.5469 - dense_1_loss: 8.9337 - conv2d_18_dice_coef: 0.5468 - conv2d_18_binary_crossentropy: 0.1549 - dense_1_dice_coef: 0.5899 - dense_1_binary_crossentropy: 8.9337
Epoch 00017: val_loss did not improve from 8.55251
141/141 [==============================] - 664s 5s/step - loss: 8.3868 - conv2d_18_loss: -0.5469 - dense_1_loss: 8.9337 - conv2d_18_dice_coef: 0.5468 - conv2d_18_binary_crossentropy: 0.1549 - dense_1_dice_coef: 0.5899 - dense_1_binary_crossentropy: 8.9337 - val_loss: 8.6040 - val_conv2d_18_loss: -0.4752 - val_dense_1_loss: 9.0792 - val_conv2d_18_dice_coef: 0.4649 - val_conv2d_18_binary_crossentropy: 0.1744 - val_dense_1_dice_coef: 0.5789 - val_dense_1_binary_crossentropy: 9.0792
Epoch 18/20
141/141 [==============================] - ETA: 0s - loss: 8.3844 - conv2d_18_loss: -0.5493 - dense_1_loss: 8.9337 - conv2d_18_dice_coef: 0.5493 - conv2d_18_binary_crossentropy: 0.1619 - den

/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2885_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/113_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2295_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2847_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2852_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/626_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/936_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-

/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2055_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/3209_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/5293_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/3921_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/867_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2220_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2654_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/sit

...

/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/1804_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/4231_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/42_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/4252_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/3198_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/127_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/693_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-p

/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/5339_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/1325_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/5289_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/1414_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/4910_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/218_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2913_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/sit

/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/4113_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/1286_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2332_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/4535_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/967_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2137_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/3698_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/sit

/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/786_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2692_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/217_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/596_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/1176_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2175_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2605_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-

/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2545_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/3862_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2529_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/903_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/1486_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/2164_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:241: UserWarning: preds/393_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site